## Qunatize Model

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import snapshot_download

# Define the repository ID (Model Name)
repo_id = "Qwen/Qwen3-4B"

local_dir = "Qwen3_4B"

# Download the snapshot
local_path = snapshot_download(
    repo_id=repo_id,
    local_dir=local_dir,
)

print(f"Model snapshot downloaded to: {local_path}")

### 16 Bit

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "/content/Qwen3_4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="auto"
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def invoke(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=3768
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    return thinking_content, content


In [ ]:
prompt = """Your are medical expert and have the understanding of all the medical procedures

Please provide an answer of my Query

Note :
  - You have to provide the answers in the two specific ways as mentioned:
  - Scientific way
  - Simple way (even any no medical person can understand)

Output format:
  - PLease generate a response in the python dictionary and it has to enclose with the code blocks astriks '```'
  - Dot provide any other irrelevent text data other than python dictionary

Query: A 65-year-old man with a 40-pack-year smoking history presents with unintentional weight loss, hyponatremia, and confusion. What is the most likely underlying endocrine mechanism behind his symptoms?


Answer:
"""

In [ ]:
thinking_content, content = invoke(prompt)


In [ ]:
print(content)


```python
{
    "scientific": "The most likely endocrine mechanism is SIADH (Syndrome of Inappropriate Antidiuretic Hormone Secretion), where excessive ADH secretion leads to water retention and dilutional hyponatremia. This can be secondary to a tumor (e.g., lung cancer) that secretes ADH, causing confusion and weight loss due to metabolic stress and tumor burden.",
    "simple": "The body is retaining too much water, making the blood too diluted (low sodium), which causes confusion. The weight loss and confusion are also due to the effects of a tumor (like lung cancer) that's causing these issues."
}
```


In [ ]:
{
    "scientific": "The most likely endocrine mechanism is SIADH (Syndrome of Inappropriate Antidiuretic Hormone Secretion), where excessive ADH secretion leads to water retention and dilutional hyponatremia. This can be secondary to a tumor (e.g., lung cancer) that secretes ADH, causing confusion and weight loss due to metabolic stress and tumor burden.",
    "simple": "The body is retaining too much water, making the blood too diluted (low sodium), which causes confusion. The weight loss and confusion are also due to the effects of a tumor (like lung cancer) that's causing these issues."
}

### CHATGPT answer:


```
{
  "scientific_way": "The most likely endocrine mechanism is ectopic secretion of antidiuretic hormone (ADH) by a small-cell lung carcinoma, leading to Syndrome of Inappropriate Antidiuretic Hormone Secretion (SIADH). Excess ADH increases renal water reabsorption, producing dilutional hyponatremia. The resulting low serum sodium causes neurological symptoms such as confusion, lethargy, and altered mental status.",
  
  "simple_way": "The symptoms are most likely caused by a type of lung cancer that makes too much of a hormone called ADH. This hormone tells the body to hold onto water. Keeping too much water makes the blood ‘watered down,’ lowering the sodium level. When sodium becomes too low, people can get confused, weak, and very sick."
}

```

> Add blockquote



### 32 bit

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "/content/Qwen3_4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float32,
    device_map="auto"
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def invoke(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=3768
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    return thinking_content, content


In [ ]:
thinking_content, content = invoke(prompt)
print(content)

```python
{
    "scientific": "The most likely underlying endocrine mechanism is Syndrome of Inappropriate Antidiuretic Hormone Secretion (SIADH), secondary to a small cell lung cancer (SCLC) tumor. SCLC, which is associated with a 40-pack-year smoking history, can secrete ADH (arginine vasopressin) or other peptides, leading to excessive water retention. This results in hyponatremia (low sodium levels) and cerebral edema, causing confusion. Unintentional weight loss is due to the metabolic effects of the tumor and cachexia.",
    "simple": "The man's symptoms are likely caused by a tumor (like lung cancer) that makes his body retain too much water. This leads to low sodium levels and confusion. The weight loss is because the tumor is using up his body's resources."
}
```


In [ ]:
pip install -q accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.8 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.48.2
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: 
Author-email: Tim Dettmers <dettmers@cs.washington.edu>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, packaging, torch
Required-by: 


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

config_q = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [ ]:
model_name = "/content/Qwen3_4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=config_q,
    device_map="cuda"
)
model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear4bit(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear4bit(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
 

In [ ]:
def invoke(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=3768
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    return thinking_content, content


In [ ]:
prompt = """Your are medical expert and have the understanding of all the medical procedures

Please provide an answer of my Query

Note :
  - You have to provide the answers in the two specific ways as mentioned:
  - Scientific way
  - Simple way (even any no medical person can understand)

Output format:
  - PLease generate a response in the python dictionary and it has to enclose with the code blocks astriks '```'
  - Dot provide any other irrelevent text data other than python dictionary

Query: A 65-year-old man with a 40-pack-year smoking history presents with unintentional weight loss, hyponatremia, and confusion. What is the most likely underlying endocrine mechanism behind his symptoms?


Answer:
"""

In [ ]:
thinking_content, content = invoke(prompt)
print(content)

```python
{
  "scientific": "The most likely endocrine mechanism is SIADH (Syndrome of Inappropriate Antidiuretic Hormone Secretion), caused by small cell lung cancer. This tumor produces ADH, leading to water retention, hyponatremia, confusion, and unintentional weight loss.",
  "simple": "The body is making too much antidiuretic hormone, causing the body to hold on to water. This lowers sodium levels, making the person confused and causing weight loss, which is often linked to cancer."
}
```


In [ ]:
```python
{
  "scientific": "The most likely endocrine mechanism is SIADH (Syndrome of Inappropriate Antidiuretic Hormone Secretion), caused by small cell lung cancer. This tumor produces ADH, leading to water retention, hyponatremia, confusion, and unintentional weight loss.",
  "simple": "The body is making too much antidiuretic hormone, causing the body to hold on to water. This lowers sodium levels, making the person confused and causing weight loss, which is often linked to cancer."
}
```


## Embedding model Test

In [ ]:
!pip install llama-index-embeddings-huggingface

In [ ]:
!pip install -q llama_index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 764.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 848.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
all_mini_v2 = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2", device="cuda", trust_remote_code=True)

In [ ]:
qwen06 = HuggingFaceEmbedding(model_name="Qwen/Qwen3-Embedding-0.6B", device="cuda", trust_remote_code=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [ ]:
gemaa = HuggingFaceEmbedding(model_name="Leonardojdss/my-embedding-gemma", device="cuda", trust_remote_code=True)

modules.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

3_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

In [ ]:
text = """
Cooking is a beautiful blend of creativity and precision, where simple ingredients transform into flavorful experiences. It’s not just about following recipes—it’s about feeling the textures, smelling the aromas, and adjusting each element to match your taste. Whether you’re sautéing vegetables, simmering spices, or baking something warm and comforting, cooking becomes a form of expression that brings both joy and nourishment.
Aerodynamics is the study of how air interacts with moving objects, shaping everything from aircraft flight to the design of cars and sports equipment. It focuses on forces like lift, drag, and thrust, and how the shape and motion of an object influence these forces. By understanding aerodynamic principles, engineers can create structures that move more efficiently through air—reducing resistance, increasing stability, and improving speed. Whether it’s a jet soaring through the sky or a cyclist cutting through the wind, aerodynamics plays a crucial role in making movement smoother and more optimized.
Quantum physics explores the strange and fascinating behavior of matter and energy at the smallest scales, where the rules of classical physics no longer apply. In this realm, particles can exist in multiple states at once, teleport information through entanglement, and behave both like waves and particles. Quantum physics helps us understand the fundamental structure of the universe and powers modern technologies like lasers, semiconductors, and quantum computers. It reveals a world that is unpredictable yet governed by beautifully complex mathematical principles, challenging our intuition about how reality works.
"""

In [ ]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)

In [ ]:
splitter_0 = SemanticSplitterNodeParser(breakpoint_percentile_threshold=85, embed_model=all_mini_v2)
splitter_1 = SemanticSplitterNodeParser(breakpoint_percentile_threshold=85, embed_model=qwen06)
splitter_2 = SemanticSplitterNodeParser(breakpoint_percentile_threshold=85, embed_model=gemaa)

In [ ]:
from llama_index.core.schema import Document

In [ ]:
docs = Document(text=text)

In [ ]:
chuck_minilm = splitter_0.get_nodes_from_documents([docs])
chuck_qwen06 = splitter_1.get_nodes_from_documents([docs])
chunk_gemma = splitter_2.get_nodes_from_documents([docs])

In [ ]:
for chunk in chuck_minilm:
    print(chunk.text)
    print("\n")



Cooking is a beautiful blend of creativity and precision, where simple ingredients transform into flavorful experiences. It’s not just about following recipes—it’s about feeling the textures, smelling the aromas, and adjusting each element to match your taste. 


Whether you’re sautéing vegetables, simmering spices, or baking something warm and comforting, cooking becomes a form of expression that brings both joy and nourishment.
Aerodynamics is the study of how air interacts with moving objects, shaping everything from aircraft flight to the design of cars and sports equipment. It focuses on forces like lift, drag, and thrust, and how the shape and motion of an object influence these forces. By understanding aerodynamic principles, engineers can create structures that move more efficiently through air—reducing resistance, increasing stability, and improving speed. 


Whether it’s a jet soaring through the sky or a cyclist cutting through the wind, aerodynamics plays a crucial role in

In [ ]:
for chunk in chuck_qwen06:
    print(chunk.text)
    print("\n")



Cooking is a beautiful blend of creativity and precision, where simple ingredients transform into flavorful experiences. It’s not just about following recipes—it’s about feeling the textures, smelling the aromas, and adjusting each element to match your taste. 


Whether you’re sautéing vegetables, simmering spices, or baking something warm and comforting, cooking becomes a form of expression that brings both joy and nourishment.
Aerodynamics is the study of how air interacts with moving objects, shaping everything from aircraft flight to the design of cars and sports equipment. It focuses on forces like lift, drag, and thrust, and how the shape and motion of an object influence these forces. By understanding aerodynamic principles, engineers can create structures that move more efficiently through air—reducing resistance, increasing stability, and improving speed. Whether it’s a jet soaring through the sky or a cyclist cutting through the wind, aerodynamics plays a crucial role in ma

In [ ]:
for chunk in chunk_gemma:
    print(chunk.text)
    print("\n")



Cooking is a beautiful blend of creativity and precision, where simple ingredients transform into flavorful experiences. It’s not just about following recipes—it’s about feeling the textures, smelling the aromas, and adjusting each element to match your taste. 


Whether you’re sautéing vegetables, simmering spices, or baking something warm and comforting, cooking becomes a form of expression that brings both joy and nourishment.
Aerodynamics is the study of how air interacts with moving objects, shaping everything from aircraft flight to the design of cars and sports equipment. It focuses on forces like lift, drag, and thrust, and how the shape and motion of an object influence these forces. By understanding aerodynamic principles, engineers can create structures that move more efficiently through air—reducing resistance, increasing stability, and improving speed. Whether it’s a jet soaring through the sky or a cyclist cutting through the wind, aerodynamics plays a crucial role in ma

## Image to Image

In [ ]:
https://huggingface.co/collections/nvidia/nemotron-rag

In [ ]:
!pip install -U -q diffusers

In [ ]:
!pip install -U -q accelerate bitsandbytes

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `model_access` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cac

In [ ]:
!pip install -q hqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
import hqq
from diffusers import Flux2Pipeline, Flux2Transformer2DModel
from transformers import AutoModel
from hqq.core.quantize import HQQLinear, BaseQuantizeConfig
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [ ]:
def replace_with_hqq(model, quant_config):
    """
    Recursively replaces nn.Linear layers with HQQLinear layers.
    This must match the exact logic used during quantization.
    """
    for name, child in model.named_children():
        if isinstance(child, torch.nn.Linear):
            # Create empty HQQ layer
            hqq_layer = HQQLinear(
                child,
                quant_config=quant_config,
                compute_dtype=torch.bfloat16,
                device="cuda",
                initialize=False
            )
            setattr(model, name, hqq_layer)
        else:
            replace_with_hqq(child, quant_config)


In [ ]:

hqq_config = BaseQuantizeConfig(
    nbits=2,
    group_size=64,
    axis=1
)

In [ ]:

model_id = "AlekseyCalvin/FLUX2_dev_2bit_hqq"

print("Loading Text Encoder (Mistral)...")
# Initialize skeleton
text_encoder = AutoModel.from_pretrained(
    "black-forest-labs/FLUX.2-dev", # Load config from base model
    subfolder="text_encoder",
    torch_dtype=torch.bfloat16
)

Loading Text Encoder (Mistral)...


config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/57.9k [00:00<?, ?B/s]

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

...

text_encoder/model-00005-of-00010.safete(…):   0%|          | 0.00/4.78G [00:00<?, ?B/s]

text_encoder/model-00006-of-00010.safete(…):   0%|          | 0.00/4.89G [00:00<?, ?B/s]

text_encoder/model-00003-of-00010.safete(…):   0%|          | 0.00/4.89G [00:00<?, ?B/s]

text_encoder/model-00008-of-00010.safete(…):   0%|          | 0.00/4.78G [00:00<?, ?B/s]

text_encoder/model-00004-of-00010.safete(…):   0%|          | 0.00/4.78G [00:00<?, ?B/s]

In [ ]:
# Swap layers
replace_with_hqq(text_encoder, hqq_config)
# Load quantized weights
te_path = hf_hub_download(model_id, filename="text_encoder/model.safetensors")
te_state_dict = load_file(te_path)
text_encoder.load_state_dict(te_state_dict)
text_encoder = text_encoder.to("cpu")


In [ ]:

print("Loading Transformer (Flux 2)...")
# Initialize skeleton
transformer = Flux2Transformer2DModel.from_pretrained(
    "black-forest-labs/FLUX.2-dev",
    subfolder="transformer",
    torch_dtype=torch.bfloat16
)

In [ ]:
# Swap layers
replace_with_hqq(transformer, hqq_config)
# Load quantized weights
tr_path = hf_hub_download(model_id, filename="transformer/diffusion_pytorch_model.safetensors")
tr_state_dict = load_file(tr_path)
transformer.load_state_dict(tr_state_dict)
transformer = transformer.to("cuda")


In [ ]:

print("Assembling Pipeline...")
pipe = Flux2Pipeline.from_pretrained(
    "black-forest-labs/FLUX.2-dev",
    transformer=transformer,
    text_encoder=text_encoder,
    torch_dtype=torch.bfloat16
)
pipe.enable_model_cpu_offload()


In [ ]:
rint("Ready for Inference!")
prompt = "A photo of a sneaky koala hiding behind book stacks at a library, calm snowy landscape visible through large window in the backdrop..."
image = pipe(prompt, guidance_scale=4, num_inference_steps=40).images[0]
image.save("KoalaTesting.png")


In [ ]:
from huggingface_hub import hf_hub_download

model_id = "city96/FLUX.2-dev-gguf"

hf_hub_download(repo_id=model_id, filename="flux2-dev-Q3_K_S.gguf")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


flux2-dev-Q3_K_S.gguf:   0%|          | 0.00/15.8G [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--city96--FLUX.2-dev-gguf/snapshots/ade5d688ddab0d9cf4a5b01bf4321e01c115020d/flux2-dev-Q3_K_S.gguf'

In [ ]:
flux_gguf_file_path = "/root/.cache/huggingface/hub/models--city96--FLUX.2-dev-gguf/snapshots/ade5d688ddab0d9cf4a5b01bf4321e01c115020d/flux2-dev-Q3_K_S.gguf" # Copy paste the actual path

In [ ]:
import torch
from diffusers import Flux2Pipeline, BitsAndBytesConfig
from diffusers.utils import load_image
import requests
import io

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [ ]:
repo_id = "diffusers/FLUX.2-dev-bnb-4bit" #quantized text-encoder and DiT. VAE still in bf16
device = "cuda"
torch_dtype = torch.bfloat16

In [ ]:
from diffusers import DiffusionPipeline
from diffusers.quantizers import PipelineQuantizationConfig

pipeline_quant_config = PipelineQuantizationConfig(
    quant_backend="bitsandbytes_4bit",
    quant_kwargs={"load_in_4bit": True, "bnb_4bit_quant_type": "nf4", "bnb_4bit_compute_dtype": torch.bfloat16},
    components_to_quantize=["transformer", "text_encoder_2"],
)

In [ ]:
pipe = DiffusionPipeline.from_pretrained(
    repo_id,
    quantization_config=pipeline_quant_config,
    torch_dtype=torch.bfloat16,
).to("cuda")

In [ ]:
prompt = """A focused software engineer sitting at a modern desk, writing Python code on a dual-monitor setup. The screens display clean, well-formatted Python scripts with functions, classes, and comments. Soft ambient lighting, a minimalistic workspace, laptop, mechanical keyboard, and a cup of coffee nearby. The engineer is deeply concentrated, with subtle reflections of code on their glasses. Futuristic tech vibe, high-resolution, ultra-realistic lighting, professional workspace atmosphere."""

In [ ]:
image = pipe(prompt).images[0]

In [ ]:
image